<a href="https://colab.research.google.com/github/icecream126/CVML_ML_Study/blob/main/2021_08_20/Hyeseong/GCN_Adversarial_Attack_with_FGSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
cd gdrive/MyDrive/Colab Notebooks/CVML/Adversarial Attack/Graph Adversarial Attack

/content/gdrive/MyDrive/Colab Notebooks/CVML/Adversarial Attack/Graph Adversarial Attack


In [4]:
!nvidia-smi

Fri Aug 20 02:52:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Dataset

## 1.1. Download `Cora` Dataset

`Cora dataset`: 2708개의 scientific publications (Citation Network) \\
* `node`: 머신러닝 논문 \\
* `edge`: 논문 쌍 간의 인용 \\
* `class`: 논문 클래스 ID(카테고리) (7개)
* `features of node`: 논문들에서 등장하는 1433개의 단어를 모아 하나의 `단어 사전`으로 만들고, 논문마다 `단어 사전`에 있는 단어들의 등장 여부를 `feature vector`로 만든다.
* `graph node classification`: 임의의 논문에 대해 **논문 내에 등장한 단어들과 인용 관계만으로 어떤 종류(`class`)의 논문인지 예측**

In [5]:
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheControl

In [6]:
pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


In [7]:
import torch
from torch_geometric.datasets import Planetoid

In [8]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

`Cora()`: 데이터셋 전체가 하나의 그래프 \\
`num_classes`: 클래스 수 (not graph but node) \\
`num_node_features`: 1433 node features

In [9]:
print(dataset)
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

Cora()
1
7
1433


`data = dataset[0]`: `slicing`으로 `node` 한 개 가져옴 \\
`train_mask`: train nodes (140 nodes) \\
`val_mask`: validation nodes (500 nodes) \\
`test_mask`: test nodes (1000 nodes)

In [10]:
data = dataset[0]
print(data)
print(data.is_undirected())
print(data.train_mask.sum().item())
print(data.val_mask.sum().item())
print(data.test_mask.sum().item())

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
True
140
500
1000


In [11]:
print(data.train_mask)
print(data.y)

tensor([ True,  True,  True,  ..., False, False, False])
tensor([3, 4, 4,  ..., 3, 3, 3])


`data.edge_index`: `[2, 10556]`, graph connectivity ? \\
`data.edge_attr`: `None`, edge feature matrix \\
`data.x`: `[2708, 1433]`, node feature matrix (feature vector) -> 단어 사전 등장 여부 (2708개의 각 논문에 1433개의 단어 여부) \\
`data.y`: `[2708]`, graph or node의 target -> 각 paper의 카테고리 ID (7개) \\
`data.train_mask`: `[2708]`, train nodes ? \\

In [14]:
print(data.edge_index)
print(data.edge_attr)
print(data.x)
print(data.y)
# print(data.pos)
# print(data.normal)
# print(data.face)
print(data.train_mask)

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
None
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([3, 4, 4,  ..., 3, 3, 3])
tensor([ True,  True,  True,  ..., False, False, False])


## 1.2. Dataloader

In [15]:
from torch_geometric.data import DataLoader

`torch_geometric.data.DataLoader`: 그래프 단위 데이터 처리

In [16]:
loader = DataLoader(dataset, batch_size=1, shuffle=True)

`batch`: 각 노드들을 각각의 batch와 mapping하는 열벡터 (1082 node에 32개의 batch mapping)\\
(ex. $[0 \ ... \ 0 \ 1 \ ... \ n-2 \ n-1 \ ... \ n-1]^T$)

In [17]:
for batch in loader:
    print(batch)
    print(batch.num_graphs)

Batch(batch=[2708], edge_index=[2, 10556], ptr=[2], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
1


# 2. Learning Model

## 2.1. Generate the `Vanilla Graph Convolutional Networks (GCN)`

In [18]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [19]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

## 2.2. Train

In [20]:
import time

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    t = time.time()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    _, pred = out.max(dim=1)
    correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    loss.backward()
    optimizer.step()

    print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss.item()),
              'acc_train: {:.4f}'.format(acc),
              'time: {:.4f}s'.format(time.time() - t))

Epoch: 0001 loss_train: 1.9401 acc_train: 1.1643 time: 0.0100s
Epoch: 0002 loss_train: 1.8190 acc_train: 2.7714 time: 0.0055s
Epoch: 0003 loss_train: 1.6793 acc_train: 3.6714 time: 0.0059s
Epoch: 0004 loss_train: 1.5425 acc_train: 4.1643 time: 0.0058s
Epoch: 0005 loss_train: 1.3657 acc_train: 4.3357 time: 0.0061s
Epoch: 0006 loss_train: 1.2540 acc_train: 4.5571 time: 0.0055s
Epoch: 0007 loss_train: 1.1116 acc_train: 4.6286 time: 0.0067s
Epoch: 0008 loss_train: 1.0340 acc_train: 4.7357 time: 0.0065s
Epoch: 0009 loss_train: 0.9301 acc_train: 4.7357 time: 0.0057s
Epoch: 0010 loss_train: 0.7849 acc_train: 5.0000 time: 0.0057s
Epoch: 0011 loss_train: 0.6761 acc_train: 4.9429 time: 0.0058s
Epoch: 0012 loss_train: 0.6225 acc_train: 5.1857 time: 0.0055s
Epoch: 0013 loss_train: 0.5344 acc_train: 5.2643 time: 0.0059s
Epoch: 0014 loss_train: 0.4699 acc_train: 5.2429 time: 0.0059s
Epoch: 0015 loss_train: 0.4323 acc_train: 5.2071 time: 0.0061s
Epoch: 0016 loss_train: 0.3330 acc_train: 5.1786 time: 

## 2.3. Evaluation

In [22]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8010


# 3. Attack

## 3.1. Generate `FGSM` Attack

* white-box attack

$x_{perturbed} = x + \epsilon \times sign(∇_xJ(θ,x,y))$

In [45]:
def fgsm(input, epsilon, data_grad):
#    return torch.clamp(input + epsilon*data_grad.sign(), 0, 1)
    return input + data_grad.sign()

## 3.2. Run Attack

In [24]:
epsilons = [0, .05, .1, .15, .2, .25, .3]

In [25]:
model.eval()

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [26]:
from torch.autograd import Variable

In [43]:
def test(model, device, loader, epsilon):

    # 정확도 카운터
    correct = 0
    adv_examples = []
    retain_graph=True

    # 테스트 셋의 모든 예제에 대해 루프를 돕니다

    # 디바이스(CPU or GPU) 에 데이터와 라벨 값을 보냅니다
    data = dataset[0].to(device)
    data.retain_graph = True

    # 텐서의 속성 중 requires_grad 를 설정합니다. 공격에서 중요한 부분입니다
    data.x.requires_grad = True

    # 데이터를 모델에 통과시킵니다
    output = model(data)
    #init_pred = output.max(1, keepdim=True)[1] # 로그 확률의 최대값을 가지는 인덱스를 얻습니다

    # 만약 초기 예측이 틀리면, 공격하지 않도록 하고 계속 진행합니다
    #if init_pred.item() != data.y[data.test_mask].item():
    #    continue

    # 손실을 계산합니다
    loss = F.nll_loss(output, data.y)

    # 모델의 변화도들을 전부 0으로 설정합니다
    model.zero_grad()

    # 후방 전달을 통해 모델의 변화도를 계산합니다
    #loss.backward()

    # 변화도 값을 모읍니다
    data_grad = torch.autograd.grad(loss, data.x)[0]

    # FGSM 공격을 호출합니다
    perturbed_data = data
    perturbed_data.x = fgsm(data.x, epsilon, data_grad)

    # 작은 변화가 적용된 이미지에 대해 재분류합니다
    output = model(perturbed_data)


    # 올바른지 확인합니다
    #final_pred = output.max(1, keepdim=True)[1] # 로그 확률의 최대값을 가지는 인덱스를 얻습니다
    #if final_pred.item() == data.y[data.test_mask].item():
    #    correct += 1
    #    # 0 엡실론 예제에 대해서 저장합니다
    #    if (epsilon == 0) and (len(adv_examples) < 5):
    #        adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
    #        adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
    #else:
        # 추후 시각화를 위하 다른 예제들을 저장합니다
    #    if len(adv_examples) < 5:
    #        adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
    #        adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )


    # 해당 엡실론에서의 최종 정확도를 계산합니다
    final_acc = correct/float(len(loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, len(loader), final_acc))

    # 정확도와 적대적 예제를 리턴합니다
    return final_acc, adv_examples

In [44]:
accuracies = []
examples = []

# 각 엡실론에 대해 테스트 함수를 실행합니다
for eps in epsilons:
    acc, ex = test(model, device, loader, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.05	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.1	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.15	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.2	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.25	Test Accuracy = 0 / 1 = 0.0
Epsilon: 0.3	Test Accuracy = 0 / 1 = 0.0
